<a href="https://colab.research.google.com/github/remidion/PGM-Project/blob/main/Ridership.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/remidion/PGM-Project/blob/main/main.ipynb)

In [1]:
# Imports
from getpass import getpass
import pandas as pd
import os
import numpy as np
import glob

In [2]:
!pip install geopandas
import geopandas as gpd

     |████████████████████████████████| 972kB 5.6MB/s 
     |████████████████████████████████| 6.5MB 32.6MB/s 
     |████████████████████████████████| 14.8MB 297kB/s 


# Data Preparation

## Download MBTA Data

In [3]:
if not os.path.exists("MBTA Bus Ridership.geojson"):
  !wget https://opendata.arcgis.com/datasets/eec03d901d2e470ebd5758c60d793e8e_0.geojson
  !mv eec03d901d2e470ebd5758c60d793e8e_0.geojson "MBTA Bus Ridership.geojson"


--2020-12-06 02:34:54--  https://opendata.arcgis.com/datasets/eec03d901d2e470ebd5758c60d793e8e_0.geojson
Resolving opendata.arcgis.com (opendata.arcgis.com)... 3.226.96.47, 3.227.94.184, 3.214.85.175, ...
Connecting to opendata.arcgis.com (opendata.arcgis.com)|3.226.96.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/json]
Saving to: ‘eec03d901d2e470ebd5758c60d793e8e_0.geojson’

eec03d901d2e470ebd5     [    <=>             ]   1.29G  16.5MB/s    in 85s     

2020-12-06 02:36:20 (15.6 MB/s) - ‘eec03d901d2e470ebd5758c60d793e8e_0.geojson’ saved [1380620043]



## Load MBTA Ridership Data

In [4]:
df = gpd.read_file("MBTA Bus Ridership.geojson")
df

,FID,season,route_id,route_variant,direction_id,trip_start_time,day_type_id,day_type_name,stop_name,stop_id,stop_sequence,boardings,alightings,load_,sample_size,geometry
0,1,Fall 2016,1,1-_-0,0,04:37:00,day_type_01,weekday,WASHINGTON ST OPP RUGGLES ST,1,2,0.4,0.3,7.8,13,None
1,2,Fall 2016,1,1-_-0,0,04:37:00,day_type_01,weekday,ALBANY ST OPP RANDALL ST,10003,5,0.1,0.0,9.4,13,None
2,3,Fall 2016,1,1-_-0,0,04:37:00,day_type_01,weekday,MASSACHUSETTS AVE @ SIDNEY ST,101,19,0.0,0.4,10.4,13,None
3,4,Fall 2016,1,1-_-0,0,04:37:00,day_type_01,weekday,MASSACHUSETTS AVE @ PROSPECT,102,20,0.6,2.5,8.5,13,None
4,5,Fall 2016,1,1-_-0,0,04:37:00,day_type_01,weekday,MASSACHUSETTS AVE @ BIGELOW S,104,21,0.1,0.2,8.4,13,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3451727,3451728,Fall 2019,99,99-8-1,1,22:30:00,day_type_02,saturday,HIGHLAND AVE @ PINE ST,5062,9,0.0,0.0,0.0,5,None
3451728,3451729,Fall 2019,99,99-8-1,1,22:30:00,day_type_02,saturday,CLIFTON ST @ DEXTER ST,5066,15,0.0,0.0,0.0,5,None
3451729,3451730,Fall 2019,99,99-8-1,1,22:30:00,day_type_02,saturday,CLIFTON ST @ CEDAR ST,5067,16,0.0,0.0,0.0,5,None
3451730,3451731,Fall 2019,99,99-8-1,1,22:30:00,day_type_02,saturday,SUMMER ST @ CLIFTON ST,5068,17,0.0,0.0,0.0,5,None


## Utility functions

In [5]:
def filter_on_row_col(df, target_rows, target_columns):
  for key, value in target_rows:
      df.drop(df[df[key] != value].index, inplace=True)
  df.drop(df.columns.difference(target_columns), axis=1, inplace=True)

def load_targets_from_csvs(target_columns, target_rows):
  bus_arrival_files = glob.glob('./*.csv')

  df_cumulative = pd.DataFrame()
  for f in bus_arrival_files:
    df_tmp = pd.read_csv(f)
    filter_on_row_col(df_tmp, target_rows, target_columns)
    df_cumulative = df_cumulative.append(df_tmp)
    print(f"Loaded {f}")
    del df_tmp

  return df_cumulative

# EDA on Routes, Ridership

In [6]:
tot_sample_size = df.groupby(by=['route_id', 'direction_id'], as_index=False)['sample_size'].sum()
tot_sample_size.sort_values(by = 'sample_size', ascending = False).head(20)

,route_id,direction_id,sample_size
94,23,1,857938
112,28,1,827974
1,1,1,758696
248,66,1,754626
120,32,1,746150
93,23,0,735042
111,28,0,724153
84,22,1,703984
119,32,0,667073
0,1,0,649238


# Preprocess data

In [7]:
# Check route variants
df[df["route_id"]=='1'].groupby(['route_variant']).size()

route_variant
1-_-0    29946
1-_-1    31732
dtype: int64

In [8]:
filter_columns = ["season", "trip_start_time", "stop_id", "stop_sequence", "boardings", "alightings", "load_", "sample_size"]
filter_rows = (df['route_id'] == '1') & \
              (df['route_variant'] == "1-_-1") & \
              (df['day_type_name'] == "weekday")
sort_columns = ['season', 'trip_start_time', 'stop_sequence']
df2 = df[filter_columns][filter_rows].sort_values(by=sort_columns)
df2

,season,trip_start_time,stop_id,stop_sequence,boardings,alightings,load_,sample_size
7491,Fall 2016,05:10:00,110,1,5.1,0.0,5.7,13
7495,Fall 2016,05:10:00,2168,2,0.5,0.0,6.5,13
7493,Fall 2016,05:10:00,2166,3,0.2,0.7,6.6,13
7494,Fall 2016,05:10:00,2167,4,0.0,0.0,6.6,13
7220,Fall 2016,05:10:00,66,5,0.2,0.0,6.8,13
...,...,...,...,...,...,...,...,...
2604889,Fall 2019,25:12:00,10100,24,0.1,0.3,2.6,36
2604890,Fall 2019,25:12:00,10101,25,0.0,0.0,2.6,36
2604897,Fall 2019,25:12:00,62,26,0.0,0.0,2.6,36
2604898,Fall 2019,25:12:00,63,27,0.0,0.1,2.5,36


In [9]:
pd.set_option('display.max_rows', 60)
pd.set_option('display.max_columns', 60)
df3 = df2.pivot(index=['season','trip_start_time'], columns=['stop_sequence'], values=['boardings',	'alightings',	'load_', 'sample_size'])
df3

boardings                                          \
stop_sequence                    1    2    3    4    5    6    7    8    9    
season    trip_start_time                                                     
Fall 2016 05:10:00              5.1  0.5  0.2  0.0  0.2  0.0  0.0  0.2  1.4   
          05:30:00              4.6  0.5  0.1  0.0  0.0  0.1  0.4  0.0  0.5   
          05:50:00              2.5  0.9  0.9  0.0  0.0  2.2  1.0  0.7  1.0   
          06:06:00              2.1  0.9  1.1  0.0  0.4  0.3  0.6  0.6  1.0   
          06:21:00              1.8  1.1  0.3  0.0  0.1  0.8  0.5  0.8  1.2   
...                             ...  ...  ...  ...  ...  ...  ...  ...  ...   
Fall 2019 24:08:00              1.8  0.5  0.4  0.2  0.4  0.2  0.1  0.3  0.2   
          24:22:00              1.2  1.3  0.2  0.1  0.1  0.1  0.0  0.3  0.2   
          24:37:00              1.8  0.7  0.3  0.1  0.1  0.0  0.0  0.1  0.1   
          24:52:00              1.6  1.2  0.1  0.1  0.1  0.1  0.0  0.0  0.1   
          25:12:00              3.2  0.0  0.0  0.0  0.2  0.0  0.0  0.0  0.2   

                                                                               \
stop_sequence                10   11   12   13   14   15   16   17    18   19   
season    trip_start_time                                                       
Fall 2016 05:10:00          2.0  0.9  1.2  0.2  0.0  0.3  1.0  2.1   3.4  0.0   
          05:30:00          5.5  0.8  0.3  0.2  0.0  1.1  1.5  0.7   4.2  0.1   
          05:50:00          8.7  2.1  0.9  0.0  0.1  4.8  1.1  0.3   5.0  0.1   
          06:06:00          6.4  1.6  0.9  0.0  0.0  5.4  0.4  1.4  13.6  0.0   
          06:21:00         10.3  1.8  0.5  0.4  0.8  6.4  1.2  3.2  13.5  0.3   
...                         ...  ...  ...  ...  ...  ...  ...  ...   ...  ...   
Fall 2019 24:08:00          1.6  1.1  0.4  1.4  0.1  1.7  0.5  0.2   1.0  0.2   
          24:22:00          1.0  0.6  0.4  0.6  0.1  0.7  0.3  0.2   0.9  0.2   
          24:37:00          1.1  0.2  0.2  0.6  0.1  0.6  0.2  0.2   0.6  0.2   
          24:52:00          0.9  0.5  0.1  0.6  0.1  0.6  0.2  0.3   0.5  0.1   
          25:12:00          0.5  0.1  0.1  0.1  0.1  0.3  0.1  0.1   0.1  0.1   

                                                                        \
stop_sequence               20   21   22   23   24   25   26   27   28   
season    trip_start_time                                                
Fall 2016 05:10:00         0.2  0.1  0.2  0.4  0.0  0.0  0.0  0.0  0.0   
          05:30:00         0.0  0.5  0.4  0.3  0.0  0.0  0.0  0.0  0.0   
          05:50:00         0.2  0.9  0.6  0.6  0.0  0.0  0.0  0.0  0.0   
          06:06:00         0.1  0.4  1.1  1.4  0.1  0.0  0.0  0.0  0.0   
          06:21:00         0.1  0.4  0.5  1.2  0.0  0.0  0.0  0.0  0.0   
...                        ...  ...  ...  ...  ...  ...  ...  ...  ...   
Fall 2019 24:08:00         0.3  0.3  0.2  0.3  0.0  0.0  0.0  0.0  0.0   
          24:22:00         0.1  0.4  0.1  0.1  0.0  0.0  0.0  0.0  0.0   
          24:37:00         0.2  0.3  0.1  0.3  0.0  0.0  0.0  0.0  0.0   
          24:52:00         0.2  0.3  0.1  0.2  0.0  0.0  0.0  0.0  0.0   
          25:12:00         0.1  0.1  0.1  0.2  0.1  0.0  0.0  0.0  0.0   

                          alightings       ... load_      sample_size        \
stop_sequence                     1    2   ...    27   28          1     2    
season    trip_start_time                  ...                                
Fall 2016 05:10:00               0.0  0.0  ...   4.5  0.1        13.0  13.0   
          05:30:00               0.0  0.1  ...   2.7  0.0        22.0  22.0   
          05:50:00               0.0  0.0  ...   6.3  0.2         9.0   9.0   
          06:06:00               0.0  0.0  ...   5.3  0.0         8.0   8.0   
          06:21:00               0.0  0.0  ...   6.4  0.0        13.0  13.0   
...                              ...  ...  ...   ...  ...         ...   ...   
Fall 2019 24:08:00               0.0

# Training

## TODO

# Inference

## TODO